# PredictionGuardRerank

>[Prediction Guard](https://predictionguard.com) is a secure, scalable GenAI platform that safeguards sensitive data, prevents common AI malfunctions, and runs on affordable hardware.

## Overview

### Integration details

## Setup
To access Prediction Guard models, contact us [here](https://predictionguard.com/get-started) to get a Prediction Guard API key and get started.


### Credentials
Once you have a key, you can set it with

In [3]:
import os

os.environ["PREDICTIONGUARD_API_KEY"] = "<Your Prediction Guard API Key>"

### Installation

In [ ]:
%pip install --upgrade --quiet langchain-predictionguard langchain langchain-community

In [ ]:
# %pip install --upgrade --quiet  faiss

# OR  (depending on Python version)

%pip install --upgrade --quiet  faiss-cpu

In [4]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## Instantiation

In [1]:
from langchain_predictionguard import PredictionGuardRerank

compressor = PredictionGuardRerank(model="bge-reranker-v2-m3")

### Set up the base vector store retriever

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_predictionguard import PredictionGuardEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader("../fixtures/state_of_the_union.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(
    texts, PredictionGuardEmbeddings(model="bge-m3")
).as_retriever(search_kwargs={"k": 20})

query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.
----------------------------------------------------------------------------------------------------
Document 3:

Madam

## Usage

In [7]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_predictionguard import PredictionGuard

llm = PredictionGuard(temperature=0)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

['One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', 'We cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.', 'As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given poten

## Use within a chain

In [5]:
from langchain.chains import RetrievalQA

In [6]:
chain = RetrievalQA.from_chain_type(
    llm=PredictionGuard(temperature=0), retriever=compression_retriever
)

In [7]:
chain({"query": query})

/var/folders/5w/yfpbg8996p98gpf6fd8hg1c00000gn/T/ipykernel_66360/3572038011.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"query": query})


{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': " The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.<|end_of_text|>://www.whitehouse.gov/briefing-room/speeches-remarks/2022/03/01/remarks-by-president-biden-on-the-state-of-the-union/<|end_of_text|>://www.whitehouse.gov/briefing-room/speeches-remarks/2022/03/01/remarks-by-president-biden-on-the-state-of-the-union/<|end_of_text|>://www.whitehouse.gov/briefing-room/speeches-remarks/2022/03/01/remarks-by-president-biden-on-the-state-of-the-union/<|end_of_text|>://www.whitehouse.gov/briefing-room/speeches-remarks/2022/03/01/remarks-by-president-biden-on-the-state-of-the-union/<|end_of_text|>://www.whitehouse.gov/briefing-room/speeches-remarks/2022/03/01/remarks-by-president-biden-on-the-state-of-the-union/<|end_of_tex

## API reference
For detailed documentation of all PredictionGuardRerank features and configurations check out the API reference: https://github.com/predictionguard/langchain-predictionguard/